# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType
from pyspark.sql.functions import desc, asc
from pyspark.sql.functions import sum as Fsum

from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark

import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [3]:
spark = SparkSession \
    .builder \
    .appName("Our first Python Spark SQL example") \
    .getOrCreate()

In [4]:
path = "./data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [5]:
user_log.select('page').where(user_log.userId == '').dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [7]:
user_log.filter(user_log.userId == '').show()

+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|artist|      auth|firstName|gender|itemInSession|lastName|length|level|location|method| page|registration|sessionId|song|status|           ts|userAgent|userId|
+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|  null|Logged Out|     null|  null|            0|    null|  null| free|    null|   PUT|Login|        null|     5598|null|   307|1513721196284|     null|      |
|  null|Logged Out|     null|  null|           26|    null|  null| paid|    null|   GET| Home|        null|      428|null|   200|1513721274284|     null|      |
|  null|Logged Out|     null|  null|            5|    null|  null| free|    null|   GET| Home|        null|     2941|null|   200|1513722009284|     null|      |
|  null|Logged Out|     null|  nul

In [8]:
user_log.filter(user_log.userId == '').groupBy('level').count().orderBy('count').show()

+-----+-----+
|level|count|
+-----+-----+
| paid|  143|
| free|  193|
+-----+-----+



In [9]:
user_log.filter(user_log.userId == '').groupBy('page').count().orderBy('count').show()

+-----+-----+
| page|count|
+-----+-----+
| Help|    8|
|About|   15|
|Login|  126|
| Home|  187|
+-----+-----+



# Question 3

How many female users do we have in the data set?

In [10]:
user_log.dropDuplicates(['userId']).filter(user_log.gender == 'F').count()

462

# Question 4

How many songs were played from the most played artist?

In [11]:
user_log.groupBy('artist').count().sort(desc('count')).show(truncate=False)

+----------------------------------------------------------------------+-----+
|artist                                                                |count|
+----------------------------------------------------------------------+-----+
|null                                                                  |1653 |
|Coldplay                                                              |83   |
|Kings Of Leon                                                         |69   |
|Florence + The Machine                                                |52   |
|BjÃÂ¶rk                                                              |46   |
|Dwight Yoakam                                                         |45   |
|Justin Bieber                                                         |43   |
|The Black Keys                                                        |40   |
|OneRepublic                                                           |37   |
|Muse                                               

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [12]:
# TODO: write your code to answer question 5
#Creating an ufd to flag Home
flag_home = udf(lambda x: 1 if x == 'Home' else 0, IntegerType())

In [14]:
# Create an dataframe with visitHome column check
user_home_visit = user_log.withColumn('visitHome', flag_home('page'))

In [21]:
#Check if worked
user_home_visit.filter(user_home_visit.page == 'Home').head()

Row(artist=None, auth='Logged In', firstName='Jordyn', gender='F', itemInSession=0, lastName='Jones', length=None, level='free', location='Syracuse, NY', method='GET', page='Home', registration=1513648531284, sessionId=1746, song=None, status=200, ts=1513720913284, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"', userId='1747', visitHome=1)

In [22]:
# Creating a window valuation
from pyspark.sql import Window
windowVal = Window.partitionBy('userId').orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)

In [26]:
cumsum = user_home_visit.filter((user_home_visit.page == 'NextSong') | (user_home_visit.page == 'Home')) \
    .select('userId', 'page', 'ts', 'visitHome') \
    .withColumn('period', Fsum('visitHome').over(windowVal))

In [28]:
cumsum.filter((cumsum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+



In [29]:
cumsum.collect()